## Computing CLEVER metric with CNN model from Keras trained with CIFAR10 dataset

In [1]:
# import neccessary files
from __future__ import absolute_import, division, print_function, unicode_literals

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout
import numpy as np
import tensorflow as tf

# art library
from art import metrics
from art.classifiers import KerasClassifier
from art.utils import load_dataset


Using TensorFlow backend.


## Preparing dataset and environment
In this file, we will use the CIFAR10 dataset.
We will disable tensorflow 2.0's eager evaluation as ART has not been updated to support eager evaluation.

In [2]:
# disable tensorflow 2.0 eager evaluation as it is not yet supported
tf.compat.v1.disable_eager_execution()
# Read CIFAR10 dataset
(x_train, y_train), (x_test, y_test), min_, max_ = load_dataset(str('cifar10'))
x_train, y_train = x_train[:5000], y_train[:5000]
x_test, y_test = x_test[:500], y_test[:500]
im_shape = x_train[0].shape

## Creating and preparing the model
For this example, we will use Keras Convolutional Neural Network model (CNN) without any adversarial defense training.

In [3]:
# Create Keras convolutional neural network - basic architecture from Keras examples
# Source here: https://github.com/keras-team/keras/blob/master/examples/cifar10_cnn.py
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


## Wrapping a whitebox classifier
This creates a classifier wrapper for training of model

In [4]:
# Create classifier wrapper
classifier = KerasClassifier(model=model, clip_values=(min_, max_))
classifier.fit(x_train, y_train, nb_epochs=10, batch_size=128)

Epoch 1/10
40/39 [==============================] - 5s 135ms/step - loss: 2.1707 - accuracy: 0.1768
Epoch 2/10
40/39 [==============================] - 4s 108ms/step - loss: 1.8425 - accuracy: 0.3229
Epoch 3/10
40/39 [==============================] - 5s 117ms/step - loss: 1.6132 - accuracy: 0.4137
Epoch 4/10
40/39 [==============================] - 4s 111ms/step - loss: 1.5153 - accuracy: 0.4471
Epoch 5/10
40/39 [==============================] - 5s 115ms/step - loss: 1.4414 - accuracy: 0.4744
Epoch 6/10
40/39 [==============================] - 4s 107ms/step - loss: 1.3433 - accuracy: 0.5090
Epoch 7/10
40/39 [==============================] - 5s 115ms/step - loss: 1.2795 - accuracy: 0.5410
Epoch 8/10
40/39 [==============================] - 4s 112ms/step - loss: 1.2278 - accuracy: 0.5641
Epoch 9/10
40/39 [==============================] - 5s 115ms/step - loss: 1.1454 - accuracy: 0.5924
Epoch 10/10
40/39 [==============================] - 4s 110ms/step - loss: 1.1161 - accuracy: 0.6057

## Computing of CLEVER score


#### Predetermined parameters
Using the authors' predetermined values, we will use the following parameters:
- nb_batches = 50
- batch_size = 10
- radius = 5
- norm = 1

In [5]:
#using CLEVER score with first 10 test samples
scores = []
for i in range(10):
    scores.append(metrics.clever_u(classifier,x_test[i],50,10,5,1))

avg_score = sum(scores)/len(scores)
print(avg_score)

1.608338990406446


## Create an adversarial attack 
we will now create an adversarial attack: Projected Gradient Descent


In [6]:
#importing packages
from art.attacks import ProjectedGradientDescent
from art.defences import AdversarialTrainer
from keras.preprocessing.image import ImageDataGenerator
from art.data_generators import KerasDataGenerator

In [7]:
# Create attack; here, we use projected gradient descent
pgd = ProjectedGradientDescent(classifier, eps=8, eps_step=2, max_iter=10, num_random_init=20)

In [8]:
# Create some adversarial samples for evaluation
x_test_pgd = pgd.generate(x_test)

## Adding defence layers based off advesarial attack
we will now add basing adversarial training defence to the model and evaluate its robustness using the CLEVER score

### Creating more data for adversarial training
as adversarial datasets from x_train will be used to evaluate the trained model, we will need to create more data for adversarial training of model.
Thus, we will use Keras image augmentation to create more data for adversarial training.

In [9]:
# Build a Keras image augmentation object and wrap it in ART
batch_size = 50
datagen = ImageDataGenerator(horizontal_flip=True, width_shift_range=0.125, height_shift_range=0.125,
                             fill_mode='constant', cval=0.)
datagen.fit(x_train)
art_datagen = KerasDataGenerator(datagen.flow(x=x_train, y=y_train, batch_size=batch_size, shuffle=True),
                                 size=x_train.shape[0], batch_size=batch_size)


### Creating and training adversarial trainer

In [13]:
# Create adversarial trainer and perform adversarial training
adv_trainer = AdversarialTrainer(classifier, attacks=pgd, ratio=1.)
adv_trainer.fit_generator(art_datagen, nb_epochs=2)

Epoch 1/1
1/1 [==============================] - 0s 43ms/step - loss: 2.2981 - accuracy: 0.1000
Epoch 1/1
1/1 [==============================] - 0s 47ms/step - loss: 2.3045 - accuracy: 0.1000
Epoch 1/1
1/1 [==============================] - 0s 66ms/step - loss: 2.3042 - accuracy: 0.0400
Epoch 1/1
1/1 [==============================] - 0s 74ms/step - loss: 2.3229 - accuracy: 0.0400
Epoch 1/1
1/1 [==============================] - 0s 81ms/step - loss: 2.2952 - accuracy: 0.0600
Epoch 1/1
1/1 [==============================] - 0s 38ms/step - loss: 2.3008 - accuracy: 0.1000
Epoch 1/1
1/1 [==============================] - 0s 60ms/step - loss: 2.3036 - accuracy: 0.1200
Epoch 1/1
1/1 [==============================] - 0s 63ms/step - loss: 2.3125 - accuracy: 0.1400
Epoch 1/1
1/1 [==============================] - 0s 77ms/step - loss: 2.3033 - accuracy: 0.1000
Epoch 1/1
1/1 [==============================] - 0s 71ms/step - loss: 2.2865 - accuracy: 0.2200
Epoch 1/1
1/1 [=========================

In [14]:
scores = []
for i in range(9):
    scores.append(metrics.clever_u(classifier,x_test[i],50,10,5,1))

avg_score = sum(scores)/len(scores)
print(avg_score)

4.140382268196472
